In [64]:
import websocket
import json
import numpy
import talib
import pandas as pd

In [65]:
cc = "ethusdt"
interval = "1m"
SOCKET =f'wss://stream.binance.com:9443/ws/{cc}@kline_{interval}'

RSI_PERIOD = 6
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
in_position = False

closes = []

In [66]:
def on_error(ws, error):
    print(error)
    
def on_close(ws):
    print('closed connection')
    
def on_message(ws, message):
    global in_position
    json_message = json.loads(message)
    candle = json_message['k']
    is_candle_closed = candle['x']
    date = pd.to_datetime(candle['t'], unit='ms')
    open1 = candle['o']
    close = candle['c']
    high = candle['h']
    low = candle['l']
    if is_candle_closed:
        print(f"Date:{date}, Open:{open1}, Close:{close}, High:{high}, Low:{low}")
        print(in_position)
        closes.append(float(close))
        print(f"closes:{closes}")
        if len(closes) >= RSI_PERIOD:
            np_closes = numpy.array(closes)
            print(f"np_closes:{np_closes}")
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            print(f'rsi:{rsi}')
            last_rsi = rsi[-1]
            print(f'last_rsi:{last_rsi}')
            if last_rsi > RSI_OVERBOUGHT:
                print("last_rsi > 70")
                if in_position:
                    print("Overbought! Sell! Sell! Sell!")
                    in_position = False
                    print(f'point:{in_position}')
                else:
                    print("It is overbought, but we don't own any. Nothing to do.")
            if last_rsi < RSI_OVERSOLD:
                print("last_rsi < 30")
                if in_position:
                    print("It is oversold, but you already own it, nothing to do.")
                else:
                    print("Oversold! Buy! Buy! Buy!")
                    in_position = True
                    print(f'point:{in_position}')
                    
ws = websocket.WebSocketApp(SOCKET, on_close=on_close, on_message=on_message, on_error=on_error)

In [67]:
ws.run_forever()

Date:2021-08-25 06:10:00, Open:3209.99000000, Close:3205.25000000, High:3210.00000000, Low:3204.00000000
False
closes:[3205.25]
Date:2021-08-25 06:11:00, Open:3205.25000000, Close:3205.18000000, High:3208.45000000, Low:3204.99000000
False
closes:[3205.25, 3205.18]
Date:2021-08-25 06:12:00, Open:3205.19000000, Close:3205.96000000, High:3207.63000000, Low:3205.18000000
False
closes:[3205.25, 3205.18, 3205.96]
Date:2021-08-25 06:13:00, Open:3205.96000000, Close:3206.14000000, High:3208.68000000, Low:3205.50000000
False
closes:[3205.25, 3205.18, 3205.96, 3206.14]
Date:2021-08-25 06:14:00, Open:3206.14000000, Close:3210.99000000, High:3211.00000000, Low:3206.13000000
False
closes:[3205.25, 3205.18, 3205.96, 3206.14, 3210.99]
Date:2021-08-25 06:15:00, Open:3210.99000000, Close:3206.52000000, High:3215.00000000, Low:3205.67000000
False
closes:[3205.25, 3205.18, 3205.96, 3206.14, 3210.99, 3206.52]
np_closes:[3205.25 3205.18 3205.96 3206.14 3210.99 3206.52]
rsi:[nan nan nan nan nan nan]
last_rs

False